In [1]:
import pandas as pd 
df_article = pd.read_csv('Ver2_article_ngram.csv')
df_bond = pd.read_csv('ver1_bond_ngram.csv')

In [2]:
df_article = df_article[['date', 'n_gram','Labelling']]

In [3]:
df_article = df_article.dropna(axis=0)

In [4]:
df_bond = df_bond.dropna(axis=0)

In [5]:
df_bond = df_bond[['date', 'n_gram','Labelling']]

In [6]:
df= pd.concat([df_bond, df_article], axis = 0, ignore_index=True) 
df

,date,n_gram,Labelling
0,2008-05-22,"['만기/NNG', '금리/NNG', '상승/NNG', '하/VV', '만기/NNG...",up
1,2008-05-22,"['만기/NNG', '금리/NNG', '하락/NNG', '하/VV', '만기/NNG...",up
2,2008-05-22,"['유가/NNG', '경제학/NNG', '고유가/NNG', '충격/NNG', '시/...",up
3,2008-05-22,"['만기/NNG', '금리/NNG', '상승/NNG', '하/VV', '만기/NNG...",up
4,2008-05-22,"['만기/NNG', '금리/NNG', '상승/NNG', '하/VV', '만기/NNG...",up
...,...,...,...
2971175,2017-12-31,"['강점/NNG', '살린/VV', '차별화/NNG', '되/VV', '비즈니스모델...",down
2971176,2017-12-31,"['위하/VV', '각자/NNG', '핵심역량/NNG', '역량/NNG', '높/V...",down
2971177,2017-12-31,"['경제/NNG', '혈맥/NNG', '금융/NNG', '산업/NNG', '역할/N...",down
2971178,2017-12-31,"['기술금융/NNG', '활성화/NNG', '모험자본/NNG', '육성/NNG', ...",down


In [7]:
from tqdm.notebook import tqdm
from collections import defaultdict

doccnt0 = 0 
doccnt1 = 0 

wordfreq = defaultdict(lambda:[0,0])

for i in tqdm(range(len(df["Labelling"]))):
    w_list = []
    label = [0 if df['Labelling'].iloc[i] == 'up' else 1][0]
    w_list = df['n_gram'][i].strip('\[').replace("\'", "").strip('\]').split(",")
    for ngram in w_list:
            wordfreq[ngram][label] += 1

In [8]:
wordfreq

defaultdict(<function __main__.<lambda>()>,
            {'만기/NNG': [7347, 13508],
             ' 금리/NNG': [286431, 388729],
             ' 상승/NNG': [139593, 197605],
             ' 하/VV': [791517, 1269661],
             ' 만기/NNG': [26433, 32883],
             ' 로/NNG': [3170, 122],
             ' 마감/NNG': [21137, 31287],
             ' 각종/NNG': [2932, 4277],
             ' 변동성/NNG': [11379, 16832],
             ' 크/VV': [16136, 27062],
             ' 널뛰기/NNG': [102, 180],
             ' 장세/NNG': [5004, 7519],
             ' 펼침/VV': [8, 10],
             ' 단기/NNG': [16383, 22132],
             ' 외채/NNG': [1589, 1886],
             ' 대책/NNG': [8044, 14254],
             ' 강구/NNG': [571, 941],
             ' 하/XSV': [364650, 556528],
             ' 뜻/NNG': [3527, 6207],
             ' 내비쳐/VV': [19, 27],
             ' 출발/NNG': [8038, 13408],
             ' 국고채/NNG': [47950, 28834],
             ' 바이백/NNG': [477, 434],
             ' 효과/NNG': [13335, 20421],
             ' 극대/NNG': [640, 9

In [9]:
# 데이터 프레임 변환 
prob = pd.DataFrame(wordfreq.values(), columns=['up_count', 'down_count'])
prob['ngram'] = pd.DataFrame(wordfreq.keys(), columns=['ngram'])
prob['total_count'] = prob['up_count'] + prob['down_count']

prob 

,up_count,down_count,ngram,total_count
0,7347,13508,만기/NNG,20855
1,286431,388729,금리/NNG,675160
2,139593,197605,상승/NNG,337198
3,791517,1269661,하/VV,2061178
4,26433,32883,만기/NNG,59316
...,...,...,...,...
168315,0,1,꼬불꼬불/MAG,1
168316,0,1,소유물/NNG,1
168317,0,1,흥인지문/NNG,1
168318,0,1,信行/NNG,1


In [10]:
# 15개 미만 제거 
rule = prob[prob['total_count']<=15].index
prob_ = prob.drop(rule)

In [11]:
# 극성 계산
haw_sum = sum(prob_['up_count'])
dov_sum = sum(prob_['down_count'])

print(haw_sum), print(dov_sum)

prob_['polar_score'] = (prob_['up_count']/haw_sum) /(prob_['down_count']/dov_sum)

16578199
24071996


In [12]:
# 극성 라벨링 
prob_['polarity'] = prob_['polar_score'].apply(lambda x: 'Hawkish' if x> 1.3 else "Dovish" if x < 0.76 else "nothing")

In [13]:
prob_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50297 entries, 0 to 167589
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   up_count     50297 non-null  int64  
 1   down_count   50297 non-null  int64  
 2   ngram        50297 non-null  object 
 3   total_count  50297 non-null  int64  
 4   polar_score  50297 non-null  float64
 5   polarity     50297 non-null  object 
dtypes: float64(1), int64(3), object(2)
memory usage: 2.7+ MB


In [14]:
prob_ 

,up_count,down_count,ngram,total_count,polar_score,polarity
0,7347,13508,만기/NNG,20855,0.789757,nothing
1,286431,388729,금리/NNG,675160,1.069911,nothing
2,139593,197605,상승/NNG,337198,1.025747,nothing
3,791517,1269661,하/VV,2061178,0.905206,nothing
4,26433,32883,만기/NNG,59316,1.167212,nothing
...,...,...,...,...,...,...
166555,41,17,물가/NNG;부진/NNG;일시적/VAX,58,3.501948,Hawkish
167046,13,8,중기부/NNG,21,2.359544,Hawkish
167318,12,8,청구자/NNG,20,2.178041,Hawkish
167357,25,2,어마/NNG,27,18.150340,Hawkish


In [15]:
df_hawkish_dict_ver4= prob_[prob_['polarity']=='Hawkish']
df_hawkish_dict_ver4 

,up_count,down_count,ngram,total_count,polar_score,polarity
5,3170,122,로/NNG,3292,37.728904,Hawkish
21,47950,28834,국고채/NNG,76784,2.414674,Hawkish
22,477,434,바이백/NNG,911,1.595892,Hawkish
45,2310,150,의/NNG,2460,22.361219,Hawkish
61,21450,23370,인플레이션/NNG,44820,1.332734,Hawkish
...,...,...,...,...,...,...
165790,12,13,파면/NNG,25,1.340333,Hawkish
166555,41,17,물가/NNG;부진/NNG;일시적/VAX,58,3.501948,Hawkish
167046,13,8,중기부/NNG,21,2.359544,Hawkish
167318,12,8,청구자/NNG,20,2.178041,Hawkish


In [16]:
df_hawkish_dict_ver4.to_csv('df_hawkish_dict_ver4.csv', index=False)

In [17]:
df_dovish_dict_ver4 = prob_[prob_['polarity']=='Dovish']
df_dovish_dict_ver4 

,up_count,down_count,ngram,total_count,polar_score,polarity
36,39641,78645,인하/NNG,118286,0.731894,Dovish
41,12,23,개입설/NNG,35,0.757579,Dovish
76,5,12,당분간/MAG;금리/NNG;하락/NNG,17,0.605011,Dovish
82,10395,28784,금리/NNG;인하/NNG,39179,0.524382,Dovish
97,22,43,교체수요/NNG,65,0.742898,Dovish
...,...,...,...,...,...,...
165722,4,25,고용/NNG;물가/NNG;기대/NNG;개선/NNG,29,0.232324,Dovish
165914,7,110,트럼프케어/NNG,117,0.092402,Dovish
165924,14,35,경제/NNG;역사적/VAX;성장/NNG;추세/NNG;확장/NNG,49,0.580811,Dovish
165925,14,35,지수/NNG;경제/NNG;역사적/VAX;성장/NNG,49,0.580811,Dovish


In [18]:
df_dovish_dict_ver4.to_csv('df_dovish_dict_ver4.csv', index=False)